In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity

# 문제에 대한 정보
problem = pd.read_csv('tag_key.csv')

In [12]:
user = pd.read_csv('userSolved.csv')

In [13]:
problem.shape

(5598, 195)

In [14]:
# 사용할 Column
# problemid, level, averageTries, tag-key
# 데이터 전처리
problem = problem.iloc[:,1:]

In [15]:
# 새로운 피벗 테이블을 사용하기 위해 Column 명을 변경
user.columns = ['userId', 'problemId']

In [17]:
# 데이터 중복행 삭제
problem = problem.drop_duplicates()
# K-Means를 하기 위해 문제 아이디를 제거
problem_no = problem.iloc[:,1:]

In [18]:
# 30개의 단계로 구성되었으므로 30개로 작성
k = 30
model = KMeans(n_clusters=k, random_state=10)
model.fit(problem_no)

KMeans(n_clusters=30, random_state=10)

In [19]:
# Cluseter의 결과를 저장
problem['cluster'] = model.fit_predict(problem_no)

In [20]:
problem.tail()

,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,bruteforcing,...,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis,cluster
5593,21837,10,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,20
5594,21855,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
5595,21917,10,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5
5596,21937,10,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
5597,21959,10,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26


In [22]:
# 사용자와 문제정보를 병합
user_problem = pd.merge(user, problem, on="problemId")

In [24]:
# 사용자와 문제에 대한 피벗 테이블을 생성
user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')

In [54]:
# 백준에는 있지만 Solved ac에는 없는 문제를 -1로 태깅
user_problem_pi.fillna(-1,inplace=True)
user_problem_pi.head()

userId,0000000000,0000064,0000sb,0004poppop,000ian,001,001dltmdals,0021,006ws,007001as,...,zzzwldy,zzzxc123,zzzz01111,zzzz465,zzzzddz,zzzzll,zzzzong123,zzzzz9887,zzzzzzzz,zzzzzzzzu2
problemId,,,,,,,,,,,,,,,,,,,,,
1000,24.0,24.0,24.0,24.0,-1.0,24.0,-1.0,24.0,24.0,24.0,...,24.0,24.0,24.0,24.0,-1.0,24.0,24.0,24.0,24.0,24.0
1001,24.0,24.0,24.0,24.0,-1.0,24.0,-1.0,-1.0,24.0,-1.0,...,24.0,24.0,24.0,24.0,-1.0,24.0,-1.0,24.0,24.0,24.0
1002,12.0,-1.0,-1.0,-1.0,-1.0,12.0,-1.0,12.0,-1.0,-1.0,...,12.0,-1.0,-1.0,12.0,-1.0,12.0,-1.0,-1.0,12.0,-1.0
1003,27.0,-1.0,-1.0,-1.0,-1.0,27.0,-1.0,27.0,27.0,-1.0,...,27.0,-1.0,-1.0,27.0,-1.0,27.0,-1.0,-1.0,-1.0,-1.0
1004,-1.0,-1.0,-1.0,-1.0,-1.0,27.0,-1.0,27.0,27.0,-1.0,...,27.0,-1.0,-1.0,-1.0,-1.0,27.0,-1.0,-1.0,-1.0,-1.0


In [28]:
# 문제간의 유사도를 구해 새로운 matiricx생성
problem_sim = cosine_similarity(user_problem_pi,user_problem_pi).argsort()[:,::-1]

In [29]:
#문제 추천의 함수
def get_recommand_problem(df, problemId, top=30):
    target_problem_index = df[df['problemId'] == problemId].index.values
    
    # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = problem_sim[target_problem_index, :top].reshape(-1)
    # 본인 제외
    sim_index = sim_index[sim_index != target_problem_index]
    
    result = df.iloc[sim_index].sort_values('level', ascending=True)[:3]
    return result

In [31]:
get_recommand_problem(problem,14491)

,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,bruteforcing,...,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis,cluster
504,15372,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
623,23971,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
306,6162,3,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25


In [80]:
#하나의 클래스로 진행
class Recommand:
    
    def __init__(self):
        # 사용자와 문제 정보를 읽어 저장
        self.problem = pd.read_csv('tag_key.csv')
        self.user = pd.read_csv('userSolved.csv')
        # 의미 없는 열 제거
        self.problem = self.problem.iloc[:,1:]
        # 새로운 피벗 테이블을 사용하기 위해 Column 명을 변경
        self.user.columns = ['userId', 'problemId']
        # 데이터 중복행 삭제
        self.problem = self.problem.drop_duplicates()
    
    def Start_KMeans(self):
        # K-Means를 하기 위해 문제 아이디를 제거
        problem_no = self.problem.iloc[:,1:]
        # Cluster는 Solved.ac의 난이도의 수로 설정
        k = 30
        # 모델을 생성
        model = KMeans(n_clusters=k, random_state=10)
        # K-Means를 실행
        model.fit(problem_no)
        self.problem['cluster'] = model.fit_predict(problem_no)
        # 사용자와 문제정보를 병합
        user_problem = pd.merge(user, problem, on="problemId")
        # 사용자와 문제에 대한 피벗 테이블을 생성
        user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')
        # 백준에는 있지만 Solved ac에는 없는 문제를 -1로 태깅
        user_problem_pi.fillna(-1,inplace=True)
        # 사용자와 문제정보를 병합
        self.problem_sim = cosine_similarity(user_problem_pi, user_problem_pi).argsort()[:,::-1]
    
    def get_recommand_problem(self, problemId, top=30):
        target_problem_index = self.problem[self.problem['problemId'] == problemId].index.values
        # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
        sim_index = self.problem_sim[target_problem_index, :top].reshape(-1)
        # 본인 제외
        sim_index = sim_index[sim_index != target_problem_index]
    
        result = self.problem.iloc[sim_index].sort_values('level', ascending=True)[:5]
        return result

In [81]:
rc = Recommand()
rc.Start_KMeans()

In [82]:
rc.get_recommand_problem(1001)

,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,bruteforcing,...,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis,cluster
0,1000,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,24
34,11283,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
5,2475,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,24
97,8723,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,24
421,11320,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
